In [1]:
# Importing pandas for data manipulation and analysis
import pandas as pd
from pandas import DataFrame  # Importing DataFrame explicitly for easier access

# Importing numpy for numerical operations
import numpy as np

# Importing modules for making HTTP requests and parsing HTML
from urllib.request import Request, urlopen  # Used to fetch web pages
from bs4 import BeautifulSoup as soup  # Used for parsing HTML content

# Importing Selenium for web automation and scraping
from selenium import webdriver  # Provides browser automation capabilities
from webdriver_manager.chrome import ChromeDriverManager  # Manages ChromeDriver installation
from selenium.webdriver.common.keys import Keys  # Allows interaction with keyboard input
from selenium.webdriver.common.by import By  # Used to locate elements on a webpage
from selenium.webdriver.support.ui import WebDriverWait  # Used for waiting until elements are present
from selenium.webdriver.support.ui import Select  # Helps interact with dropdown elements
from selenium.webdriver.support import expected_conditions as EC  # Provides conditions for element interactions
from selenium.webdriver.chrome.options import Options  # Allows configuring ChromeDriver options

# Importing time module for adding delays in script execution
import time

In [ ]:
# File path for CSV to read in patient data
PATIENT_DATA_PATH = ""

# Username to log into patient portal
USERNAME = ""

# Password to log into patient portal
PASSWORD = ""

# URL to login into patient portal
LOGIN_URL = 'https://commerce.health.state.ny.us/doh2/applinks/contsubs/SelectManualEntryType.action?submitterInfo.submitterType=02'

In [2]:
# Utility methods
def wait(numSeconds):
    """Pauses execution for the given number of seconds."""
    time.sleep(numSeconds)

def hitEnter(element):
    """Simulates pressing the 'Enter' key on a web element."""
    element.send_keys(Keys.RETURN)

def isNaN(num):
    """Checks if a given value is NaN (Not a Number)."""
    return num != num 

def removeSlashes(dob):
    """Removes slashes from a date string."""
    return dob.replace('/', '')

def capitalizeFirst(name):
    """Capitalizes the first letter of each word in a name."""
    return name.title()

def toFullGender(gender):
    """Converts gender from 'M' or 'F' to numeric representation (1 for Male, 2 for Female)."""
    return "1" if gender == 'M' else "2"

def splitNDC(ndc):
    """Splits an NDC (National Drug Code) string by dashes into a list."""
    return ndc.split('-')


In [3]:
def login():
    """Logs into the system by entering the username and password, then submitting the form."""
    username_field = driver.find_element(By.ID, 'username')  # Locate username input field
    password_field = driver.find_element(By.ID, 'password')  # Locate password input field
    
    NAME = 'levmik20'  # Hardcoded username (consider securing this)
    PW = 'zaqxsw123'   # Hardcoded password (consider securing this)
    
    username_field.send_keys(NAME)  # Enter username
    password_field.send_keys(PW)  # Enter password
    hitEnter(password_field)  # Press 'Enter' to submit the form

def findElemAndType(id_elem, string):
    """Finds an element by its ID and types a given string into it."""
    elem_to_find = driver.find_element(By.ID, id_elem)  # Locate element
    elem_to_find.send_keys(string)  # Type the given string

def findElemAndSelect(id_elem, val):
    """Finds a dropdown element by its ID and selects an option by value."""
    elem_to_find = Select(driver.find_element(By.ID, id_elem))  # Locate dropdown element
    elem_to_find.select_by_value(val)  # Select option based on value

def findElemAndClick(id_elem):
    """Finds an element by its ID and clicks it."""
    elem_to_find = driver.find_element(By.ID, id_elem)  # Locate element
    elem_to_find.click()  # Click the element

def getDfVal(id_elem):
    """Extracts the third part of an ID string split by underscores; returns 'ndc' if an error occurs."""
    try:
        return id_elem.split('_')[2]  # Extract third segment of ID
    except: 
        return 'ndc'  # Return default value if extraction fails

def enterPatientInfo(data):
    """Fills out patient information by iterating through a list of data entries."""
    for x in range(len(data)):
        curr_id = data[x]['elem_id']  # Get element ID
        curr_type = data[x]['elem_type']  # Get element type (text, checkbox, or select)
        curr_val = data[x]['value']  # Get input value
    
        # Special handling for date of birth (DOB)
        if curr_id == 'ManualEntryPractitioner_patientInfo_strDob':
            if curr_val[0] != '1':  # Ensure DOB starts with '1' or add leading zero
                curr_val = '0' + curr_val
            findElemAndType(curr_id, curr_val)  # Enter the DOB
            
        else:     
            # Determine interaction based on element type
            if curr_type == 'c':  # Checkbox
                findElemAndClick(curr_id)
            elif curr_type == 's':  # Dropdown/select
                findElemAndSelect(curr_id, curr_val)
            else:  # Text input
                findElemAndType(curr_id, curr_val)

def clearPatientInfo(data):
    """Clears patient information fields by setting their values to an empty string."""
    for x in range(len(data)):
        curr_id = data[x]['elem_id']  # Get element ID
        curr_type = data[x]['elem_type']  # Get element type
        curr_val = data[x]['value']  # Get input value
        
        # Special handling for date of birth (DOB)
        if curr_id == 'ManualEntryPractitioner_patientInfo_strDob':
            if curr_val[0] != '1':  # Ensure DOB starts with '1' or add leading zero
                curr_val = '0' + curr_val
            findElemAndType(curr_id, '')  # Clear the DOB field
            
        else:     
            # Clear text fields
            if curr_type == 't':
                findElemAndType(curr_id, '')  # Set text input field to an empty string

In [ ]:
# List of HTML element IDs corresponding to form fields
html_ids = [
    'ManualEntryPractitioner_submitterInfo_dea',  # DEA number field
    'prescriptionInfo.reportingStatus-1',  # Checkbox for reporting status
    'patientInfo.species-1',  # Checkbox for patient species
    'ManualEntryPractitioner_patientInfo_lastName',  # Patient's last name
    'ManualEntryPractitioner_patientInfo_firstName',  # Patient's first name
    'ManualEntryPractitioner_patientInfo_address',  # Patient's address
    'ManualEntryPractitioner_patientInfo_city',  # Patient's city
    'ManualEntryPractitioner_patientInfo_state',  # Dropdown for patient's state
    'ManualEntryPractitioner_patientInfo_zipcode',  # Patient's ZIP code
    'ManualEntryPractitioner_patientInfo_strDob',  # Patient's date of birth
    'ManualEntryPractitioner_patientInfo_gender',  # Dropdown for gender selection
    'ManualEntryPractitioner_prescriptionInfo_strDateFilled',  # Date prescription was filled
    'ManualEntryPractitioner_prescriptionInfo_pharmacyRxNum',  # Pharmacy prescription number
    'ManualEntryPractitioner_prescriptionInfo_metricQty',  # Quantity of medication prescribed
    'ManualEntryPractitioner_prescriptionInfo_drugDosageUnits',  # Dropdown for dosage units
    'ManualEntryPractitioner_prescriptionInfo_daysSupply',  # Number of days supply
    'ManualEntryPractitioner_prescriptionInfo_practitionerDea',  # Prescribing practitioner's DEA number
    'ndc5', 'ndc4', 'ndc2',  # National Drug Code (NDC) values
    'ManualEntryPractitioner_prescriptionInfo_paymentType'  # Dropdown for payment type
]

# List of input types corresponding to each HTML element
# 't' = Text input, 'c' = Checkbox, 's' = Select/Dropdown
input_types = [
    't',  # DEA number (text)
    'c',  # Reporting status (checkbox)
    'c',  # Patient species (checkbox)
    't',  # Last name (text)
    't',  # First name (text)
    't',  # Address (text)
    't',  # City (text)
    's',  # State (select dropdown)
    't',  # ZIP code (text)
    't',  # Date of birth (text)
    's',  # Gender (select dropdown)
    't',  # Date filled (text)
    't',  # Pharmacy Rx number (text)
    't',  # Metric quantity (text)
    's',  # Drug dosage units (select dropdown)
    't',  # Days supply (text)
    't',  # Practitioner DEA (text)
    't', 't', 't',  # NDC fields (text)
    's'  # Payment type (select dropdown)
]

# Initialize an empty list to store input field details
inputs = []

# Loop through each element and create a dictionary with its properties
for x in range(len(html_ids)):
    curr_obj = {
        'elem_id': html_ids[x],  # Assign the HTML ID
        'elem_type': input_types[x],  # Assign the input type
        'value': ''  # Default value is an empty string
    }
    inputs.append(curr_obj)  # Append the object to the list


In [ ]:
# READ IN DATA

# Load patient data from a CSV file into a Pandas DataFrame
df = pd.read_csv(PATIENT_DATA_PATH)

# Create a new list to store combined addresses
addresses = []

# Iterate through ADDR1 and ADDR2 columns to create a full address field
for add1, add2 in zip(df['ADDR1'], df['ADDR2']):
    combined_add = ''

    # Check if ADDR2 is NaN (missing)
    if isNaN(add2):  
        combined_add = str(add1)  # Use only ADDR1 if ADDR2 is missing
    else:
        combined_add = str(add1) + ' ' + str(add2)  # Combine ADDR1 and ADDR2

    addresses.append(combined_add)  # Append the full address to the list

# Add new full address variable and clean dataframe 
# This includes:
# - Capitalizing names
# - Converting gender initials to full values
# - Changing data types of numerical variables
# - Splitting NDC codes by dashes
df['full_address'] = addresses  # Add full_address column to DataFrame

# Drop unnecessary columns: 'Unnamed: 0', 'ADDR1', and 'ADDR2'
df = df.drop('Unnamed: 0', axis=1)
df = df.drop('ADDR1', axis=1)
df = df.drop('ADDR2', axis=1)

# Capitalize first letter of first and last names
df['FIRSTNAME'] = df['FIRSTNAME'].apply(capitalizeFirst)
df['LASTNAME'] = df['LASTNAME'].apply(capitalizeFirst)

# Convert gender abbreviations ('M', 'F') to full numeric values ('1', '2')
df['SEX'] = df['SEX'].apply(toFullGender)

# Split NDC code into multiple parts by dashes
df['NDC'] = df['NDC'].apply(splitNDC)

# Convert ZIPCODE, WEEKS, and Units columns to string data type
df['ZIPCODE'] = df['ZIPCODE'].astype(str)
df['WEEKS'] = df['WEEKS'].astype(str)
df['Units'] = df['Units'].astype(str)

# Rename columns for consistency and readability
col_names = [
    'firstName', 'lastName', 'gender', 'strDob', 'city', 'state', 'zipcode', 
    'office', 'strDateFilled', 'pharmacyRxNum', 'daysSupply', 'metricQty', 'ndc', 'address'
]
df.columns = col_names  # Assign new column names


In [ ]:
# Initialize an empty list to store input objects for each row of data
input_objs = []

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    curr_row = df.iloc[index]  # Get the current row as a Pandas Series
    
    # Convert the Series to a single-row DataFrame for easier manipulation
    curr_df = pd.DataFrame([curr_row.tolist()], columns=curr_row.index)
    print(curr_df)  # Print the DataFrame for debugging

    # Define the template for form input fields
    curr_inputs = [
        {'elem_id': 'ManualEntryPractitioner_submitterInfo_dea', 'elem_type': 't', 'value': ''},
        {'elem_id': 'prescriptionInfo.reportingStatus-1', 'elem_type': 'c', 'value': ''},
        {'elem_id': 'patientInfo.species-1', 'elem_type': 'c', 'value': ''},
        {'elem_id': 'ManualEntryPractitioner_patientInfo_lastName', 'elem_type': 't', 'value': ''},
        {'elem_id': 'ManualEntryPractitioner_patientInfo_firstName', 'elem_type': 't', 'value': ''},
        {'elem_id': 'ManualEntryPractitioner_patientInfo_address', 'elem_type': 't', 'value': ''},
        {'elem_id': 'ManualEntryPractitioner_patientInfo_city', 'elem_type': 't', 'value': ''},
        {'elem_id': 'ManualEntryPractitioner_patientInfo_state', 'elem_type': 's', 'value': ''},
        {'elem_id': 'ManualEntryPractitioner_patientInfo_zipcode', 'elem_type': 't', 'value': ''},
        {'elem_id': 'ManualEntryPractitioner_patientInfo_strDob', 'elem_type': 't', 'value': ''},
        {'elem_id': 'ManualEntryPractitioner_patientInfo_gender', 'elem_type': 's', 'value': ''},
        {'elem_id': 'ManualEntryPractitioner_prescriptionInfo_strDateFilled', 'elem_type': 't', 'value': ''},
        {'elem_id': 'ManualEntryPractitioner_prescriptionInfo_pharmacyRxNum', 'elem_type': 't', 'value': ''},
        {'elem_id': 'ManualEntryPractitioner_prescriptionInfo_metricQty', 'elem_type': 't', 'value': ''},
        {'elem_id': 'ManualEntryPractitioner_prescriptionInfo_drugDosageUnits', 'elem_type': 's', 'value': ''},
        {'elem_id': 'ManualEntryPractitioner_prescriptionInfo_daysSupply', 'elem_type': 't', 'value': ''},
        {'elem_id': 'ManualEntryPractitioner_prescriptionInfo_practitionerDea', 'elem_type': 't', 'value': ''},
        {'elem_id': 'ndc5', 'elem_type': 't', 'value': ''},
        {'elem_id': 'ndc4', 'elem_type': 't', 'value': ''},
        {'elem_id': 'ndc2', 'elem_type': 't', 'value': ''},
        {'elem_id': 'ManualEntryPractitioner_prescriptionInfo_paymentType', 'elem_type': 's', 'value': ''}
    ]

    # Populate the 'value' field for relevant input elements
    for x in range(3, len(curr_inputs)):  # Skip the first three elements (DEA, reporting status, species)
        curr_id = getDfVal(curr_inputs[x]['elem_id'])  # Extract relevant field name from elem_id

        # Assign default or mapped values based on the field type
        if curr_id == 'drugDosageUnits' or curr_id == 'paymentType':
            curr_inputs[x]["value"] = "01"  # Default value for dropdown fields
        elif curr_id == 'practitionerDea':
            curr_inputs[x]["value"] = "BG7067831"  # Hardcoded DEA number
        elif curr_id in ["ndc5", "ndc4", "ndc2"]:
            curr_inputs[x]["value"] = curr_df[curr_id][0]  # Extract NDC values from the DataFrame
        else:
            curr_inputs[x]["value"] = curr_df[curr_id][0]  # Assign the corresponding DataFrame value
            
    # Append the populated input list for this row to the main list
    input_objs.append(curr_inputs)

In [ ]:
# Path to the ChromeDriver executable (Ensure this path is correct)
PATH = 'C:\Program Files (x90)\chromedriver.exe'

# Set Chrome options for the WebDriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-gpu')  # Disable GPU acceleration (useful for headless mode)
chrome_options.add_argument('--disable-dev-shm-usage')  # Prevents crashes in Docker environments
chrome_options.add_argument('--no-sandbox')  # Bypasses sandbox security restrictions
chrome_options.add_argument('--incognito')  # Launches Chrome in Incognito mode

# Define HTTP headers to mimic a real browser request
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
}

# Initialize the Chrome WebDriver with the specified options
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)

# Delete all cookies before starting a new session
driver.delete_all_cookies()

# URL for the login page
URL = 'https://commerce.health.state.ny.us/doh2/applinks/contsubs/SelectManualEntryType.action?submitterInfo.submitterType=02'

# Open the webpage in the browser
driver.get(LOGIN_URL)

# Log into the application using the predefined login function
login()

# Loop through the list of input objects to enter patient information
for x in range(len(input_objs)):
    enterPatientInfo(input_objs[x])  # Fill in patient information
    wait(3)  # Pause to allow form submission and prevent errors
    findElemAndClick('ManualEntryPractitioner_ManualEntryReset')  # Click reset button to clear form
    wait(3)  # Pause before proceeding to the next entry
    findElemAndClick('submitterInfo.submitterType-2')  # Select the submitter type option
    findElemAndClick('ManualEntryType_0')  # Click to initiate a new entry
